***GENERATED CODE FOR upsamplinggbc PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = transformationDF.drop('TransactionID')
        transformationDF = transformationDF.drop('TransactionTypeID')
        transformationDF = transformationDF.drop('DateOfService')
        transformationDF = transformationDF.drop('paid_amount')
        transformationDF = transformationDF.drop('PostDate')
        transformationDF = transformationDF.drop('PayerID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7852', 'mean': '86223.38', 'stddev': '22104.35', 'min': '11012', 'max': 'RN001', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'}], 'feature': 'CptModifier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7852', 'mean': '6.92', 'stddev': '19.06', 'min': '0000', 'max': 'RT', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CptModifier'}, {'feature_label': 'CptModifier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CptModifier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7852', 'mean': '', 'stddev': '', 'min': 'A41.9', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7852', 'mean': '', 'stddev': '', 'min': 'A49.9', 'max': 'Z98.890', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7852', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'U', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7852', 'mean': '', 'stddev': '', 'min': 'CAST APPLICATION', 'max': 'Spine And Pelvis  Xrays', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '7852', 'mean': '', 'stddev': '', 'min': 'Nurse Practitioner NP', 'max': 'Unspecified', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = transformationDF.drop('CloseDate')
        transformationDF = transformationDF.drop('MonthOfService')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split


def gradientboostingclassifier(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    model_gbm_sklearn = GradientBoostingClassifier(**optuna_parameters)
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    model_gbm_sklearn.fit(X_train, y_train)
    display(" Accuracy of Model : %s" %
            model_gbm_sklearn.score(X_test, y_test))

    data = {'model': model_gbm_sklearn,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run upsamplinggbcHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'url': '/FileStore/platform/sampleData/5fec671cea55d2e6d2ca8bb1/part-00000-tid-7652812065673519684-62cab04b-574f-444b-84df-025a0543050e-6-1-c000.csv', 'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_OAGR', 'optionalDB': 'NextGen_OAGR', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT * FROM \r\n(\r\n SELECT DISTINCT \r\n cd.ChargeDetailID, \r\n t.TransactionID, \r\n t.TransactionTypeID,\r\n cd.DateOfService, \r\n cd.Amount, \r\n t.Amount as paid_amount, \r\n t.PostDate, \r\n t.PayerID,\r\n cd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal, \r\n cptc.CPTCode, \r\n CASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier, \r\n dc.DiagnosisCode as pdiagnosis1,\r\n dc2.DiagnosisCode as pdiagnosis2,\r\nIsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END),\r\nis_authorized=(SELECT Case WHEN (apa.auth_code IS NOT NULL) THEN 1 ELSE 0 END),\r\n predetermination_ind=(SELECT Case WHEN (apa.predetermination_ind IS NOT NULL) THEN 1 ELSE 0 END),\r\n med_necessity_req_ind=(SELECT Case WHEN (apa.med_necessity_req_ind IS NOT NULL) THEN 1 ELSE 0 END),\r\nFLOOR(DATEDIFF(DAY,  p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n d.DepartmentCategory, \r\n s.SpecialityCategory,\r\n label = '1',\r\n   t.CloseDate,\r\n  MONTH(cd.DateOfService) AS MonthOfService\r\n from ChargeDetail cd\r\n inner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\n left outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.person_id and apa.payer_id = e.PayerID\r\n inner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\n inner join TransactionReason tr WITH (NOLOCK) on tr.TransactionID = t.TransactionID and tr.ChargeDetailID = t.ChargeDetailID\r\n inner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\n inner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n inner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n inner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n inner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n inner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n inner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n   inner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n WHERE\r\n cd.DateOfService between '1/01/2020'\r\n and '10/30/2020'\r\n and cd.Amount >.01 and t.CloseDate is not null and trc.reasonType = 'Denial' and t.TransactionTypeID = 1\r\n \r\n UNION\r\n SELECT DISTINCT \r\n cd.ChargeDetailID, \r\n t.TransactionID, \r\n t.TransactionTypeID,\r\n cd.DateOfService, \r\n cd.Amount, \r\n t.Amount as paid_amount, \r\n t.PostDate, \r\n t.PayerID,\r\n cd.PatientBal, cd.InsuranceBal, cd.PrimaryInsuranceBal, \r\n cptc.CPTCode, \r\n CASE WHEN cd.CPTModifier IS NOT NULL THEN cd.CPTModifier ELSE '0000' END as CptModifier,\r\n dc.DiagnosisCode as pdiagnosis1,\r\n dc2.DiagnosisCode as pdiagnosis2,\r\nIsBillable=(SELECT Case WHEN e.IsBillable = 'true' THEN 1 ELSE 0 END),\r\nis_authorized=(SELECT Case WHEN (apa.auth_code IS NOT NULL) THEN 1 ELSE 0 END),\r\n predetermination_ind=(SELECT Case WHEN (apa.predetermination_ind IS NOT NULL) THEN 1 ELSE 0 END),\r\n med_necessity_req_ind=(SELECT Case WHEN (apa.med_necessity_req_ind IS NOT NULL) THEN 1 ELSE 0 END),\r\nFLOOR(DATEDIFF(DAY,  p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\n d.DepartmentCategory, \r\n s.SpecialityCategory,\r\n  label = '0',\r\n  t.CloseDate,\r\n  MONTH(cd.DateOfService) AS MonthOfService\r\n from ChargeDetail cd\r\n inner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\n left outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.person_id and apa.payer_id = e.PayerID\r\n inner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\n inner join TransactionReason tr WITH (NOLOCK) on tr.TransactionID = t.TransactionID and tr.ChargeDetailID = t.ChargeDetailID\r\n inner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\n inner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n inner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n inner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n inner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n inner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n inner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n   inner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n WHERE\r\n cd.DateOfService between '01/01/2020'\r\n and '10/30/2020'\r\n and cd.Amount >.01 and t.CloseDate is not null and trc.reasonType != 'Denial' and t.TransactionTypeID = 1) as DND", 'dbtype': 'mssql', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run upsamplinggbcHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "7852", "mean": "", "stddev": "", "min": "00061C4A-9A47-4AE7-AF4B-173698E1456D", "max": "FFED3A9D-8115-4706-A071-3B138139A8F7", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"feature_label": "TransactionID", "transformation_label": "String Indexer"}], "feature": "TransactionID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "7852", "mean": "", "stddev": "", "min": "000581B0-0437-453D-8A8C-F0C9B83A79FF", "max": "FFFD5786-24A9-4205-8398-952FF1CE62C9", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TransactionID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TransactionTypeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "7852", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "TransactionTypeID"}, {"transformationsData": [{"feature_label": "DateOfService", "transformation_label": "Extract Date"}], "feature": "DateOfService", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "DateOfService"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "7852", "mean": "463.45", "stddev": "1305.59", "min": "10.0", "max": "16254.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "paid_amount", "type": "real", "selected": "False", "replaceby": "mean", "stats": {"count": "7852", "mean": "264.01", "stddev": "1030.3", "min": "-8050.64", "max": "13741.95", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "paid_amount"}, {"transformationsData": [{"feature_label": "PostDate", "transformation_label": "Extract Date"}], "feature": "PostDate", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "PostDate"}, {"transformationsData": [{"feature_label": "PayerID", "transformation_label": "String Indexer"}], "feature": "PayerID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "7852", "mean": "", "stddev": "", "min": "01294CB3-5FFC-486A-AA4B-6D866F7AFCC6", "max": "FEE9B52E-00AE-4C56-9BCF-00899A963994", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PatientBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "7852", "mean": "4.5", "stddev": "62.45", "min": "-501.23", "max": "3500.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PatientBal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "7852", "mean": "17.86", "stddev": "273.5", "min": "-1338.44", "max": "8223.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "InsuranceBal"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PrimaryInsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "7852", "mean": "18.3", "stddev": "272.72", "min": "-296.06", "max": "8223.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PrimaryInsuranceBal"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7852", "mean": "86223.38", "stddev": "22104.35", "min": "11012", "max": "RN001", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "CptModifier", "transformation_label": "String Indexer"}], "feature": "CptModifier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7852", "mean": "6.92", "stddev": "19.06", "min": "0000", "max": "RT", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CptModifier"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7852", "mean": "", "stddev": "", "min": "A41.9", "max": "Z98.890", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7852", "mean": "", "stddev": "", "min": "A49.9", "max": "Z98.890", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IsBillable", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7852", "mean": "0.95", "stddev": "0.21", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "IsBillable"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "is_authorized", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7852", "mean": "0.42", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "is_authorized"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "predetermination_ind", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7852", "mean": "0.42", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "predetermination_ind"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "med_necessity_req_ind", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7852", "mean": "0.42", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "med_necessity_req_ind"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7852", "mean": "57.0", "stddev": "17.04", "min": "3", "max": "97", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7852", "mean": "", "stddev": "", "min": "F", "max": "U", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7852", "mean": "", "stddev": "", "min": "CAST APPLICATION", "max": "Spine And Pelvis  Xrays", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "7852", "mean": "", "stddev": "", "min": "Nurse Practitioner NP", "max": "Unspecified", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "label", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "7852", "mean": "0.1", "stddev": "0.3", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "label"}, {"transformationsData": [{"feature_label": "CloseDate", "transformation_label": "Extract Date"}], "feature": "CloseDate", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "CloseDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MonthOfService", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "7852", "mean": "5.78", "stddev": "3.03", "min": "1", "max": "10", "missing": "0"}, "updatedLabel": "MonthOfService"}, {"feature": "CPTCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "7852", "mean": "23.95", "stddev": "43.86", "min": "0.0", "max": "311.0", "missing": "0"}, "updatedLabel": "CPTCode_stringindexer_tra..."}, {"feature": "CptModifier_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "7852", "mean": "1.71", "stddev": "3.08", "min": "0.0", "max": "30.0", "missing": "0"}, "updatedLabel": "CptModifier_stringindexer..."}, {"feature": "pdiagnosis1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "7852", "mean": "133.46", "stddev": "225.59", "min": "0.0", "max": "1096.0", "missing": "0"}, "updatedLabel": "pdiagnosis1_stringindexer..."}, {"feature": "pdiagnosis2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "7852", "mean": "123.96", "stddev": "210.92", "min": "0.0", "max": "1053.0", "missing": "0"}, "updatedLabel": "pdiagnosis2_stringindexer..."}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "7852", "mean": "0.42", "stddev": "0.49", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "7852", "mean": "1.85", "stddev": "2.65", "min": "0.0", "max": "17.0", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}, {"feature": "SpecialityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "7852", "mean": "0.68", "stddev": "0.77", "min": "0.0", "max": "5.0", "missing": "0"}, "updatedLabel": "SpecialityCategory_string..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run upsamplinggbcHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=gradientboostingclassifier(autofe, ["Amount", "PatientBal", "InsuranceBal", "PrimaryInsuranceBal", "CPTCode_stringindexer", "CptModifier_stringindexer", "pdiagnosis1_stringindexer", "pdiagnosis2_stringindexer", "IsBillable", "is_authorized", "predetermination_ind", "med_necessity_req_ind", "Age", "Gender_stringindexer", "DepartmentCategory_stringindexer", "SpecialityCategory_stringindexer"], "label")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
